# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
from azureml.core import Workspace, Experiment
from azureml.train.sklearn import SKLearn 
from azureml.train.hyperdrive.run import PrimaryMetricGoal 
from azureml.train.hyperdrive.policy import BanditPolicy 
from azureml.train.hyperdrive.sampling import RandomParameterSampling 
from azureml.train.hyperdrive.runconfig import HyperDriveConfig 
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
from azureml.widgets import RunDetails 

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
ws = Workspace.from_config()
experiment_name = 'Hyper-drive-experiment'
experiment=Experiment(ws, experiment_name)

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

from azureml.core.compute_target import ComputeTargetException 

# Choose a name for your CPU cluster 
cpu_cluster_name = "notebook142329"

# Verify that cluster does not exist already 
try: 
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.') 
except ComputeTargetException: 
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', 
                                                           max_nodes=4) 
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config) 

cpu_cluster.wait_for_completion(show_output=True)

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [ ]:
# Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1) 

# Create the different params that you will be using during training
param_sampling = RandomParameterSampling({ 
    "--learning_rate": uniform(0.01, 0.3),
    "--epochs": choice(4, 8, 16, 32),
    "--neurons": choice(48, 60, 72, 84, 96)}) 
                              

# Create your estimator and hyperdrive config
estimator = SKLearn(source_directory=".", compute_target=cpu_cluster, entry_script='entry.py', pip_packages=['azureml-dataprep', 'tensorflow']) 

hyperdrive_run_config = HyperDriveConfig(estimator=estimator,  
                                     hyperparameter_sampling=param_sampling,  
                                     policy=early_termination_policy,  
                                     primary_metric_name='accuracy',  
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,  
                                     max_total_runs=64, 
                                     max_concurrent_runs=2)

In [ ]:
# Submit your experiment
hdr = experiment.submit(config=hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(hdr).show()

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
best_run = hdr.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()

print('Best run id:', best_run.id) 
print('\n Accuracy:', best_run_metrics['accuracy'])  
print('\n Learning rate:', best_run_metrics[r'learning_rate'])  
print('\n Number of neurons in hidden layer', best_run_metrics[r'neurons'])

In [ ]:
#TODO: Save the best model


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration


model = Model.register(workspace=ws,
                       model_name='my-sklearn-model',                # Name of the registered model in your workspace.
                       model_path='./sklearn_regression_model.pkl',  # Local file to upload and register as a model.
                       model_framework=Model.Framework.SCIKITLEARN,  # Framework used to create the model.
                       resource_configuration=ResourceConfiguration(cpu=1, memory_in_gb=0.5),
                       description='Ridge regression model to predict diabetes progression.',
                       tags={'area': 'diabetes', 'type': 'regression'})

print('Name:', model.name)
print('Version:', model.version)

In [ ]:
hyperdrive_model = best_run.register_model(model_name="", model_path="./model.pkl")

from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core import Environment

environment = Environment('my-environment')

service_name = 'Heart-failure-prediction'

inference_config = InferenceConfig(entry_script='score.py', environment=environment)
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

service = Model.deploy(workspace=ws,
                      name=service_name,
                      models=[model],
                      inference_config=inference_config,
                      deployment_config=aci_config,
                      overwrite=True)
service.wait_for_deployment(show_output=True)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
import requests
import json

# URL for the web service, should be similar to:
# 'http://8530a665-66f3-49c8-a953-b82a2d312917.eastus.azurecontainer.io/score'
scoring_uri = ''
# If the service is authenticated, set the key or token
key = ''

# Two sets of data to score, so we get two results back
data = {"data":
        [
          {
            "age": 17,
            "anaemia": 1,
            "creatinine_phosphokinase": 600,
            "diabetes": 1,
            "ejection_fraction": 30,
            "high_blood_pressure": 0,
            "platelets": 263000,
            "serum_creatinine": 1.2,
            "serum_sodium": 130,
            "sex": 1,
            "smoking": 0,
            "time": 15,
          },
          {
            "age": 35,
            "anaemia": 0,
            "creatinine_phosphokinase": 500,
            "diabetes": 0,
            "ejection_fraction": 30,
            "high_blood_pressure": 0,
            "platelets": 280000,
            "serum_creatinine": 2.1,
            "serum_sodium": 150,
            "sex": 0,
            "smoking": 0,
            "time": 10,
          },
      ]
    }
# Convert to JSON string
input_data = json.dumps(data)
with open("data.json", "w") as _f:
    _f.write(input_data)

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.json())

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
service.get_logs(num_lines=100)
# service.delete()

# AmlCompute.delete(cpu_cluster)